In [1]:
# import torch_geometric
import sys
import pickle
sys.path.insert(0, '../src')

In [2]:
import tqdm
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
import torchvision
import json
import argparse
# import cv2
import numpy as np
import torch
from torch.autograd import Function
from torchvision import models
import torch.nn as nn
import os
import matplotlib.pyplot as plt
import time
from torchvision.datasets import CocoDetection


from PIL import Image
import pandas as pd

sys.path.insert(0, '../src/data/cocoapi/PythonAPI')
from pycocotools.coco import COCO

from graph_network import GCNNet
from torch_geometric.data import DataLoader as TG_DataLoader

from dataset_class import *
from etl import *

In [3]:
from PIL import Image

In [4]:
### TODO: add target transform to convert labels to proper labels using id_dict

In [5]:
train_dir = '/datasets/COCO-2017/train2017'
val_dir = '/datasets/COCO-2017/val2017'

train_ann = '/datasets/COCO-2017/anno2017/instances_train2017.json'
val_ann = '/datasets/COCO-2017/anno2017/instances_val2017.json'

In [6]:
import pickle

In [7]:
# with open('../src/data/id_dict.pkl', 'wb') as f:
#     pickle.dump(id_dict, f)    

In [8]:
with open('../src/data/id_dict.pkl', 'rb') as f:
    id_dict = pickle.load(f)

In [9]:
def convert_ids(target):
    return id_dict[target]

In [10]:
# # %%capture
# train_coco, train_data = load_data(train_ann)
# %time drop_null_annotations(coco=train_coco, data=train_data, dataDir='/datasets/COCO-2017', dataType='train2017',\
#                             annFile=train_ann, overwrite=True, tmpDataDir='../data/temp');

In [11]:
# del train_coco
# del train_data

# del val_coco
# del val_data

In [12]:
clean_train_ann = '../data/temp/annotations/clean_instances_train2017.json'
clean_val_ann = '../data/temp/annotations/clean_instances_val2017.json'

In [13]:
%time train_set = cocoDataset(root=train_dir,\
                      annotation=clean_train_ann,\
                      transforms=get_transform(),\
                      target_transform=convert_ids)

%time val_set = cocoDataset(root=val_dir,\
                      annotation=clean_val_ann,\
                      transforms=get_transform(),\
                      target_transform=convert_ids)

loading annotations into memory...
Done (t=17.55s)
creating index...
index created!
CPU times: user 16.6 s, sys: 2.44 s, total: 19 s
Wall time: 18.9 s
loading annotations into memory...
Done (t=0.65s)
creating index...
index created!
CPU times: user 606 ms, sys: 81.9 ms, total: 688 ms
Wall time: 686 ms


In [14]:
# ## Params cell
num_epoch = 10
k = 3
# fname = f'graph_80_{num_epoch}_{k}_resnet50'
train_batch_size = 12

In [15]:
trainloader = torch.utils.data.DataLoader(train_set,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=8,
                                          collate_fn=collate_fn)

valloader = torch.utils.data.DataLoader(val_set,
                                        batch_size=train_batch_size,
                                        shuffle=False,
                                        num_workers=8,
                                        collate_fn=collate_fn)

In [16]:
# Defining our model for GCN embeddings
model = models.resnet50(pretrained=False)
state_dict=torch.load("../src/models/COCO_80_10_resnet50.pth")
model.fc = nn.Linear(2048, 80)
model.load_state_dict(state_dict)

model.fc=nn.Flatten()

In [17]:
model=model.cuda()

In [18]:
from torch_geometric.data import Data
import tqdm
from torch_cluster import knn_graph

In [19]:
Idx=[]
Z=[]
XY=[]
Y=[]
with torch.no_grad():
    model.eval()
    for i,(imgs,xy,y,idx) in tqdm.tqdm(enumerate(trainloader),total=len(trainloader.dataset)//trainloader.batch_size):
#         print(imgs.shape,xy.shape,y.shape)
#         edge_index=knn_graph(xy,k=5)
        imgs=imgs.cuda()
        z=model(imgs).cpu()
        Z.append(z)
        XY.append(xy)
        Y.append(y)
        Idx.append(idx)
        
#         graphs.append(Data(x=z,pos=xy,edge_index=edge_index,y=y))
    #     print(graphs[i].x.shape)
#         if i==100: break
        del imgs, z, xy, y, idx

9773it [12:26, 13.09it/s]                            


In [20]:
del trainloader

In [21]:
Z=torch.cat(Z,0)
XY=torch.cat(XY,0)
Y=torch.cat(Y).flatten()
# Idx=torch.cat(Idx).flatten().numpy()

In [22]:
Idx=np.hstack(Idx).flatten()

In [23]:
Idx

array([101160,  36811,  36811, ...,  74517,  74517,  74517])

In [24]:
print(f'k = {k}')

k = 3


In [25]:
train_graphs=[]

for idx in tqdm.tqdm(np.unique(Idx)):
    select=(Idx==idx)
    z=Z[select]
    xy=XY[select]
    y=Y[select]
    edge_index=knn_graph(xy,k=k)
    train_graphs.append(Data(x=z,pos=xy,edge_index=edge_index,y=y))

100%|██████████| 117266/117266 [16:26<00:00, 118.82it/s]


In [26]:
val_graphs=[]
Idx=[]
Z=[]
XY=[]
Y=[]
with torch.no_grad():
    model.eval()
    for i,(imgs,xy,y,idx) in tqdm.tqdm(enumerate(valloader),total=len(valloader.dataset)//valloader.batch_size):
#         print(imgs.shape,xy.shape,y.shape)
#         edge_index=knn_graph(xy,k=5)
        imgs=imgs.cuda()
        z=model(imgs).cpu() # the 2048-dimensional embeddings provided by resnet
        Z.append(z)
        XY.append(xy)
        Y.append(y)
        Idx.append(idx)
        
#         graphs.append(Data(x=z,pos=xy,edge_index=edge_index,y=y))
    #     print(graphs[i].x.shape)
#         if i==100: break
        del imgs

413it [00:34, 11.92it/s]                         


In [27]:
Z=torch.cat(Z,0)
XY=torch.cat(XY,0)
Y=torch.cat(Y).flatten()
Idx=np.hstack(Idx).flatten()#torch.cat(Idx).flatten().numpy()

In [28]:
for idx in tqdm.tqdm(np.unique(Idx)):
    select=(Idx==idx)
    z=Z[select]
    xy=XY[select]
    y=Y[select]
    edge_index=knn_graph(xy,k=k)
    val_graphs.append(Data(x=z,pos=xy,edge_index=edge_index,y=y))

100%|██████████| 4952/4952 [00:12<00:00, 381.26it/s]


In [29]:
del valloader

In [30]:
k

3

In [31]:

mod_select = input('COCO or resnet50 graph embeddings? (coco/resnet50) ')
# if mod_select == 'coco':
    

COCO or resnet50 graph embeddings? (coco/resnet50) coco


In [32]:
%%time 
### For writing new train graphs
pickle_name = f'../data/temp/{mod_select}_train_graphs_k_{k}.pkl'
ans = input(f'Are you sure you want to overwrite {pickle_name}? ([y],n)')
if ans == 'y' or ans=='':
    with open(pickle_name, 'wb') as f:
    #     val_graphs = pickle.load(f)
        pickle.dump(train_graphs, f)

Are you sure you want to overwrite ../data/temp/coco_train_graphs_k_3.pkl? ([y],n)y
CPU times: user 28.8 s, sys: 13.4 s, total: 42.2 s
Wall time: 1min 14s


In [41]:
%%time 
### For writing new val graphs
val_pickle_name = f'../data/temp/{mod_select}_val_graphs_k_{k}.pkl'

ans = input(f'Are you sure you want to overwrite {val_pickle_name}? ([y],n)')
if ans == 'y' or ans =='':
    with open(f'../data/temp/{mod_select}_val_graphs_k_{k}.pkl', 'wb') as f:
    #     val_graphs = pickle.load(f)
        pickle.dump(val_graphs, f)

Are you sure you want to overwrite ../data/temp/coco_val_graphs_k_3.pkl? ([y],n)y
CPU times: user 1.17 s, sys: 436 ms, total: 1.61 s
Wall time: 5.1 s


In [24]:
%%time 
### For reading train_graphs
with open(f'../data/temp/{mod_select}_train_graphs_k_{k}.pkl', 'rb') as f:
    train_graphs = pickle.load(f)
#     pickle.dump(val_graphs, f)

FileNotFoundError: [Errno 2] No such file or directory: '../data/temp/coco_train_graphs_k_3.pkl'

In [66]:
%%time 
### For reading val_graphs
with open(f'../data/temp/{mod_select}_val_graphs_k_{k}.pkl', 'rb') as f:
    val_graphs = pickle.load(f)
#     pickle.dump(val_graphs, f)

CPU times: user 1.83 s, sys: 412 ms, total: 2.24 s
Wall time: 2.23 s


In [42]:
from torch_geometric.data import DataLoader as TG_DataLoader
# https://pytorch.org/docs/stable/data.html#torch.utils.data.Subset
# Batch size should be the same as batch size for dataloader above
graph_trainloader = TG_DataLoader(train_graphs, batch_size=train_batch_size, shuffle=True)

graph_valloader=TG_DataLoader(val_graphs,batch_size=train_batch_size,shuffle=True)#[0]

In [43]:
# https://github.com/jlevy44/WSI-GTFE/blob/master/notebooks/3_fit_gnn_model.ipynb
import torch, torch.nn as nn
from torch_geometric.nn import GCNConv, GATConv, DeepGraphInfomax, SAGEConv
from torch_geometric.nn import DenseGraphConv
from torch_geometric.utils import to_dense_batch, to_dense_adj, dense_to_sparse
from torch_geometric.nn import GINEConv
from torch_geometric.utils import dropout_adj
import torch.nn.functional as F



class GCNNet(torch.nn.Module):
    def __init__(self, inp_dim, out_dim, hidden_topology=[32,64,128,128], p=0.2, p2=0.0, drop_each=True):
        super(GCNNet, self).__init__()
        self.out_dim=out_dim
        self.convs = nn.ModuleList([GATConv(inp_dim, hidden_topology[0])]+[GATConv(hidden_topology[i],hidden_topology[i+1]) for i in range(len(hidden_topology[:-1]))])
        self.drop_edge = lambda edge_index: dropout_adj(edge_index,p=p2)[0]
        self.dropout = nn.Dropout(p)
        self.fc = nn.Linear(hidden_topology[-1], out_dim)
        self.drop_each=drop_each

    def forward(self, x, edge_index, edge_attr=None, return_attention=False):
        attention_weights=[]
        for conv in self.convs:
            if self.drop_each and self.training: edge_index=self.drop_edge(edge_index)
            x, attention = conv(x, edge_index, edge_attr,return_attention_weights=True)
            x = F.relu(x)
            attention_weights.append(attention)
        if self.training:
            x = self.dropout(x)
        x = self.fc(x)
        if return_attention: return x, attention_weights
        return x

In [44]:
model_gcn=GCNNet(2048,80, [32]*2)
model_mlp=nn.Sequential(nn.Sequential(nn.Linear(2048,32),nn.ReLU(),nn.Dropout(p=0.3)),nn.Linear(32,80))

In [45]:
# model_gcn.cuda()

In [46]:
# If there are GPUs, choose the first one for computing. Otherwise use CPU.
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# print(device)
# If 'cuda:0' is printed, it means GPU is available.

In [39]:
# model_gcn.to(device)

In [47]:
# Params Cell
k = 3
epochs = 10
fname = f'{mod_select}_graph_80_{epochs}_{k}'
batch_size=12
fname

'coco_graph_80_10_3'

In [48]:
### TODO: implement edge index as part of dataloader

In [49]:
loss_func = nn.CrossEntropyLoss()
opt = optim.SGD(model_gcn.parameters(), lr=0.0001, momentum=0.9)
avg_losses = []
print_freq = 100

In [50]:
for epoch in range(epochs):
    running_loss = 0.0
    for i, G in enumerate(graph_trainloader, 0):
        y=G.y#.cuda()
        z=G.x#.cuda()
        edge_index=G.edge_index#.cuda()
        
        opt.zero_grad()
        
        outputs = model_gcn(z, edge_index)
        loss = loss_func(outputs, y)
        
        loss.backward()
        
        opt.step()
        
        running_loss += loss.item()
        if i % print_freq == print_freq - 1:
            avg_loss = running_loss / print_freq
            print(f'[epoch: {epoch}, i: {i}] avg mini-batch loss: {avg_loss}')
            avg_losses.append(avg_loss)
            running_loss = 0.0
        
#         y_pred=model_gcn(z,edge_index)
#         print(y_pred.argmax(1))

[epoch: 0, i: 99] avg mini-batch loss: 4.307699842453003
[epoch: 0, i: 199] avg mini-batch loss: 4.132729833126068
[epoch: 0, i: 299] avg mini-batch loss: 3.881756374835968
[epoch: 0, i: 399] avg mini-batch loss: 3.745609750747681
[epoch: 0, i: 499] avg mini-batch loss: 3.6838740944862365
[epoch: 0, i: 599] avg mini-batch loss: 3.5968668389320375
[epoch: 0, i: 699] avg mini-batch loss: 3.5303693866729735
[epoch: 0, i: 799] avg mini-batch loss: 3.448225076198578
[epoch: 0, i: 899] avg mini-batch loss: 3.579165117740631
[epoch: 0, i: 999] avg mini-batch loss: 3.4674167132377622
[epoch: 0, i: 1099] avg mini-batch loss: 3.486649694442749
[epoch: 0, i: 1199] avg mini-batch loss: 3.4736986637115477
[epoch: 0, i: 1299] avg mini-batch loss: 3.4105195665359496
[epoch: 0, i: 1399] avg mini-batch loss: 3.431011264324188
[epoch: 0, i: 1499] avg mini-batch loss: 3.4430779242515563
[epoch: 0, i: 1599] avg mini-batch loss: 3.3659687352180483
[epoch: 0, i: 1699] avg mini-batch loss: 3.3959333658218385

[epoch: 1, i: 4299] avg mini-batch loss: 2.5003641128540037
[epoch: 1, i: 4399] avg mini-batch loss: 2.4629008948802946
[epoch: 1, i: 4499] avg mini-batch loss: 2.400014326572418
[epoch: 1, i: 4599] avg mini-batch loss: 2.410565044879913
[epoch: 1, i: 4699] avg mini-batch loss: 2.4378290355205534
[epoch: 1, i: 4799] avg mini-batch loss: 2.365341703891754
[epoch: 1, i: 4899] avg mini-batch loss: 2.394202609062195
[epoch: 1, i: 4999] avg mini-batch loss: 2.4019194424152372
[epoch: 1, i: 5099] avg mini-batch loss: 2.421831772327423
[epoch: 1, i: 5199] avg mini-batch loss: 2.399269113540649
[epoch: 1, i: 5299] avg mini-batch loss: 2.4347802114486696
[epoch: 1, i: 5399] avg mini-batch loss: 2.358822467327118
[epoch: 1, i: 5499] avg mini-batch loss: 2.4424258947372435
[epoch: 1, i: 5599] avg mini-batch loss: 2.4279038345813753
[epoch: 1, i: 5699] avg mini-batch loss: 2.358283050060272
[epoch: 1, i: 5799] avg mini-batch loss: 2.4015484285354614
[epoch: 1, i: 5899] avg mini-batch loss: 2.37876

[epoch: 2, i: 8399] avg mini-batch loss: 2.03564910531044
[epoch: 2, i: 8499] avg mini-batch loss: 1.9354020404815673
[epoch: 2, i: 8599] avg mini-batch loss: 1.9911705684661865
[epoch: 2, i: 8699] avg mini-batch loss: 1.9675495088100434
[epoch: 2, i: 8799] avg mini-batch loss: 1.9777649545669556
[epoch: 2, i: 8899] avg mini-batch loss: 1.9084068095684052
[epoch: 2, i: 8999] avg mini-batch loss: 1.946078221797943
[epoch: 2, i: 9099] avg mini-batch loss: 1.9295246481895447
[epoch: 2, i: 9199] avg mini-batch loss: 2.0082895481586456
[epoch: 2, i: 9299] avg mini-batch loss: 2.0147061729431153
[epoch: 2, i: 9399] avg mini-batch loss: 1.953300586938858
[epoch: 2, i: 9499] avg mini-batch loss: 1.96865358710289
[epoch: 2, i: 9599] avg mini-batch loss: 1.955186778306961
[epoch: 2, i: 9699] avg mini-batch loss: 1.9583187973499299
[epoch: 3, i: 99] avg mini-batch loss: 1.9426151347160339
[epoch: 3, i: 199] avg mini-batch loss: 1.921932727098465
[epoch: 3, i: 299] avg mini-batch loss: 1.933357857

[epoch: 4, i: 2799] avg mini-batch loss: 1.8215906023979187
[epoch: 4, i: 2899] avg mini-batch loss: 1.8678562581539153
[epoch: 4, i: 2999] avg mini-batch loss: 1.8725179934501648
[epoch: 4, i: 3099] avg mini-batch loss: 1.815916486978531
[epoch: 4, i: 3199] avg mini-batch loss: 1.8539721810817718
[epoch: 4, i: 3299] avg mini-batch loss: 1.7543614399433136
[epoch: 4, i: 3399] avg mini-batch loss: 1.852257866859436
[epoch: 4, i: 3499] avg mini-batch loss: 1.8542136323451996
[epoch: 4, i: 3599] avg mini-batch loss: 1.8633446204662323
[epoch: 4, i: 3699] avg mini-batch loss: 1.8079235422611237
[epoch: 4, i: 3799] avg mini-batch loss: 1.8311863923072815
[epoch: 4, i: 3899] avg mini-batch loss: 1.808873325586319
[epoch: 4, i: 3999] avg mini-batch loss: 1.8342027258872986
[epoch: 4, i: 4099] avg mini-batch loss: 1.785571244955063
[epoch: 4, i: 4199] avg mini-batch loss: 1.835044709444046
[epoch: 4, i: 4299] avg mini-batch loss: 1.8360327255725861
[epoch: 4, i: 4399] avg mini-batch loss: 1.83

[epoch: 5, i: 6899] avg mini-batch loss: 1.753891065120697
[epoch: 5, i: 6999] avg mini-batch loss: 1.7379262781143188
[epoch: 5, i: 7099] avg mini-batch loss: 1.7742386114597322
[epoch: 5, i: 7199] avg mini-batch loss: 1.7722557199001312
[epoch: 5, i: 7299] avg mini-batch loss: 1.7477998077869414
[epoch: 5, i: 7399] avg mini-batch loss: 1.7786516487598418
[epoch: 5, i: 7499] avg mini-batch loss: 1.7142262440919875
[epoch: 5, i: 7599] avg mini-batch loss: 1.7220972275733948
[epoch: 5, i: 7699] avg mini-batch loss: 1.7444044148921967
[epoch: 5, i: 7799] avg mini-batch loss: 1.715189447402954
[epoch: 5, i: 7899] avg mini-batch loss: 1.7804766237735747
[epoch: 5, i: 7999] avg mini-batch loss: 1.7837084794044495
[epoch: 5, i: 8099] avg mini-batch loss: 1.8175872242450715
[epoch: 5, i: 8199] avg mini-batch loss: 1.746301839351654
[epoch: 5, i: 8299] avg mini-batch loss: 1.7640241992473602
[epoch: 5, i: 8399] avg mini-batch loss: 1.775923582315445
[epoch: 5, i: 8499] avg mini-batch loss: 1.7

[epoch: 7, i: 1299] avg mini-batch loss: 1.698062517642975
[epoch: 7, i: 1399] avg mini-batch loss: 1.703765195608139
[epoch: 7, i: 1499] avg mini-batch loss: 1.710321877002716
[epoch: 7, i: 1599] avg mini-batch loss: 1.6580429768562317
[epoch: 7, i: 1699] avg mini-batch loss: 1.7256973588466644
[epoch: 7, i: 1799] avg mini-batch loss: 1.7022554171085358
[epoch: 7, i: 1899] avg mini-batch loss: 1.7277014076709747
[epoch: 7, i: 1999] avg mini-batch loss: 1.6560822880268098
[epoch: 7, i: 2099] avg mini-batch loss: 1.7368002420663833
[epoch: 7, i: 2199] avg mini-batch loss: 1.694552129507065
[epoch: 7, i: 2299] avg mini-batch loss: 1.699389825463295
[epoch: 7, i: 2399] avg mini-batch loss: 1.6362179279327393
[epoch: 7, i: 2499] avg mini-batch loss: 1.7324692869186402
[epoch: 7, i: 2599] avg mini-batch loss: 1.7006403923034668
[epoch: 7, i: 2699] avg mini-batch loss: 1.6490208280086518
[epoch: 7, i: 2799] avg mini-batch loss: 1.6945633137226104
[epoch: 7, i: 2899] avg mini-batch loss: 1.70

[epoch: 8, i: 5399] avg mini-batch loss: 1.6967119765281677
[epoch: 8, i: 5499] avg mini-batch loss: 1.6355060565471649
[epoch: 8, i: 5599] avg mini-batch loss: 1.663219717144966
[epoch: 8, i: 5699] avg mini-batch loss: 1.666926744580269
[epoch: 8, i: 5799] avg mini-batch loss: 1.6498251080513
[epoch: 8, i: 5899] avg mini-batch loss: 1.6758922064304351
[epoch: 8, i: 5999] avg mini-batch loss: 1.6720499140024185
[epoch: 8, i: 6099] avg mini-batch loss: 1.6790328031778337
[epoch: 8, i: 6199] avg mini-batch loss: 1.6544289255142213
[epoch: 8, i: 6299] avg mini-batch loss: 1.6335422939062119
[epoch: 8, i: 6399] avg mini-batch loss: 1.6430239856243134
[epoch: 8, i: 6499] avg mini-batch loss: 1.6638167417049408
[epoch: 8, i: 6599] avg mini-batch loss: 1.6022581189870835
[epoch: 8, i: 6699] avg mini-batch loss: 1.6838260984420776
[epoch: 8, i: 6799] avg mini-batch loss: 1.6944376361370086
[epoch: 8, i: 6899] avg mini-batch loss: 1.681558530330658
[epoch: 8, i: 6999] avg mini-batch loss: 1.659

[epoch: 9, i: 9499] avg mini-batch loss: 1.6218756085634232
[epoch: 9, i: 9599] avg mini-batch loss: 1.5879722094535829
[epoch: 9, i: 9699] avg mini-batch loss: 1.6736328828334808


In [51]:
PATH = f'../src/models/{fname}.pth'
torch.save(model_gcn.state_dict(), PATH)#, _use_new_zipfile_serialization=False)
print(f'Model saved at {PATH}')

Model saved at ../src/models/coco_graph_80_10_3.pth


**Avg Loss Curves**

In [61]:
model_3 = list(pd.read_csv('../data/out/avg_losses_80_10_resnet50.csv', usecols=['0']).values.flatten())

array([3.74180901, 3.30108178, 3.18726127, 3.09314609, 3.05046857,
       2.94124481, 2.89932525, 2.91138922, 2.81965742, 2.67611179,
       2.68950353, 2.63067372, 2.54789731, 2.52701   , 2.42434521,
       2.41313235, 2.3006439 , 2.22819111, 2.16117792, 2.18587913,
       2.17130347, 2.08066894, 1.99673106, 2.01004496, 1.96495304,
       1.94440446, 1.90609401, 1.80269321, 1.79330174, 1.82854661,
       1.76738272, 1.7724109 , 1.72534122, 1.65268592, 1.69215985,
       1.66554974, 1.67863773, 1.6349474 , 1.66041947, 1.62797758,
       1.5917943 , 1.59531796, 1.56180202, 1.55780528, 1.57831382,
       1.50112767, 1.52480217, 1.47037713, 1.44180689, 1.48018624,
       1.44026997, 1.45638546, 1.48901882, 1.40855195, 1.3793916 ,
       1.41695464, 1.37725395, 1.39097082, 1.40737379, 1.37033001,
       1.37807844, 1.38463991, 1.43878521, 1.32859889, 1.42404164,
       1.40432096, 1.33226773, 1.36402865, 1.34744722, 1.32151281,
       1.28552666, 1.35426873, 1.30215849, 1.31525838, 1.32692

In [62]:
len(avg_losses)

970

In [63]:
pd.Series(avg_losses).to_csv(f'../data/out/avg_losses_{fname}.csv')

In [74]:
# model.state_dict()

In [44]:
# for epoch in epochs:
#     for G in graph_dataloader:
#         y=G.y.cuda()
#         z=G.x.cuda()
#         edge_index=G.edge_index.cuda()
#         y_pred=model_gcn(z,edge_index)
#         print(y_pred.argmax(1))